In [ ]:
!pip install transformers


In [ ]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
csv_path = '/content/drive/MyDrive/data/text_patentability_data.csv'

train_df=pd.read_csv(csv_path)
train_df=train_df.iloc[ :5,:]
#train_df=train_df.sample(frac = 1)
batch_size=1
num_of_batches=len(train_df)/batch_size

In [ ]:
train_df.head()

,summary,detail,claims,status
0,BACKGROUND OF THE INVENTION 1. Field of Invent...,DESCRIPTION OF EXAMPLE EMBODIMENTS An architec...,1. A distributed data processing system for co...,ISS
1,FIELD OF THE INVENTION The invention relates t...,DETAILED DESCRIPTION OF THE INVENTION The pres...,1-41. (Cancelled) 42. An isolated recombinant ...,ISS
2,FIELD OF THE INVENTION The invention relates t...,DETAILED DESCRIPTION OF THE INVENTION The pres...,1-41. (Cancelled) 42. An isolated recombinant ...,ISS
3,BACKGROUND OF THE INVENTION 1. Field of the In...,DETAILED DESCRIPTION In the following descript...,1. A method comprising: providing a first comm...,ISS
4,TECHNICAL FIELD OF THE INVENTION This inventio...,DETAILED DESCRIPTION OF THE INVENTION Referrin...,1. A method for transferring information betwe...,ISS


In [ ]:
import numpy as np

np.shape(train_df)

(5, 4)

In [ ]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base',
                                             return_dict=True)
#moving the model to GPU
model.to(dev)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [ ]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [ ]:
import pandas as pd
from torch.utils.data import Dataset

class TextToTextDataset(Dataset):
    def __init__(self, tokenizer, filepath, input_column, target_column, max_token_len=512):
        self.tokenizer = tokenizer
        self.data = pd.read_csv(filepath)
        self.input_texts = self.data[input_column]
        self.target_texts = self.data[target_column]
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.input_texts[idx]
        target_text = self.target_texts[idx]

        # Tokenize input text
        input_tokens = self.tokenizer.encode_plus(input_text, max_length=self.max_token_len,
                                                   truncation=True, padding="max_length",
                                                   return_tensors="pt")

        # Tokenize target text
        target_tokens = self.tokenizer.encode_plus(target_text, max_length=self.max_token_len,
                                                    truncation=True, padding="max_length",
                                                    return_tensors="pt")

        # Input IDs and attention masks for the model
        inputs = {
            "input_ids": input_tokens["input_ids"].flatten(),
            "attention_mask": input_tokens["attention_mask"].flatten()
        }

        # Target IDs, ignoring the first token (usually <s> token)
        # We do not need attention mask for targets as they are used as labels
        targets = target_tokens["input_ids"].flatten()

        return inputs, targets

In [ ]:
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=512): # adjust max length depending on tokenizer
        self.data_frame = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length
        # self.label_map = label_map
        # redefine self.data_frame for integer valued labels
        # self.data_frame['status'] = self.data_frame['status'].map(self.label_map)

    def __len__(self):
        return self.data_frame.shape[0]

    def __getitem__(self, idx):
        text = self.data_frame.iloc[idx]['detail']
        label = self.data_frame.iloc[idx]['claims']

        # Encoding the text using the BERT tokenizer
        text_encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        label_encoding = self.tokenizer.encode_plus(
            label,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        source_ids = text_encoding["input_ids"].squeeze()
        source_mask = text_encoding["attention_mask"].squeeze()
        target_ids = label_encoding["input_ids"].squeeze()
        target_mask = label_encoding["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }

In [ ]:
train_data = TextDataset(csv_path, tokenizer)
# val_data = TextDataset(val_csv, tokenizer)

In [ ]:
train_data[0]

{'source_ids': tensor([  309, 25067, 26017,  9562,  3347,   262,     4, 15837,  3765,   262,
          4633,  7039,   196, 11810,   134,   389,  4648,    19,  2569,   270,
            77,    24,  4951, 21286,  1459,     6,  3487, 17127,     6,    11,
          2491,  9328,    21,  2323,    13,  2547, 12311,  2699,    16,  4896,
             6,  2839,  8258,     5,    71,     3, 24079,     6,  8308,  1229,
            13, 15741,   372,  2491, 15293,    11,  3487,  6002, 11573,    19,
           990,    11,  3030,    16,   490,    97,     5,     3, 22917,     6,
          2323,    33, 10431,    13,  3173, 10164,    53,  3487,    11,    87,
           127,  1055,  2491,  9328,  4089,     5, 19148,     3, 17282,  2097,
            33,  8152,    12,   482, 20218,  1413,    18, 11650,  2416,    13,
         27804,   859,  4145,    11,  1438,     5,   100,  1147,   795,     3,
             9,  4732,    21,  1288,    18, 22955,  1357,   492,    11,  2491,
          1459,    57,   936,    18,  

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]

        if _ % 10 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [ ]:
import torch
from torch.utils.data import DataLoader
#from torch.optim import Adafactor

train_dataloader = DataLoader(train_data, batch_size=1, shuffle=True)
criterion = torch.nn.CrossEntropyLoss()
'''optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)'''
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
epoch = 1
train(epoch, tokenizer, model, dev, train_dataloader, optimizer)

NameError: name 'training_logger' is not defined